# Download the dataset

In [1]:
import os
from zipfile import ZipFile
PATH_TO_ZIP = os.path.join("..", "dataset")
PATH_TO_CSV = os.path.join("..", "dataset")
with ZipFile(os.path.join(PATH_TO_ZIP, "archive.zip"), "r") as z_obj:
    z_obj.extractall(PATH_TO_CSV)

In [2]:
import pandas as pd
spam_ham_df = pd.read_csv(os.path.join(PATH_TO_CSV, "spam_or_not_spam.csv"))

# Exploratory Data Analysis

## Quick Glance

In [13]:
spam_ham_df.head(5) # 0 is ham; 1 is spam

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0


In [5]:
spam_ham_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   email   2999 non-null   object
 1   label   3000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 47.0+ KB


## Check for null values

In [10]:
spam_ham_df.isnull()

,email,label
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
2995,False,False
2996,False,False
2997,False,False
2998,False,False


In [12]:
null_mask = spam_ham_df.isnull()
rows_with_nulls = spam_ham_df[null_mask.any(axis=1)]
rows_with_nulls

,email,label
2966,NaN,1


In [14]:
# remove the row with null value for email
spam_ham_df = spam_ham_df.dropna()

In [16]:
spam_ham_df.isnull().sum()

email    0
label    0
dtype: int64

## Text Data Exploration

In [23]:
spam_ham_df.loc[:, ('email_length')] = spam_ham_df['email'].apply(len)
spam_ham_df

,email,label,email_length
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0,1522
1,martin a posted tassos papadopoulos the greek ...,0,643
2,man threatens explosion in moscow thursday aug...,0,1509
3,klez the virus that won t die already the most...,0,1015
4,in adding cream to spaghetti carbonara which ...,0,771
...,...,...,...
2995,abc s good morning america ranks it the NUMBE...,1,561
2996,hyperlink hyperlink hyperlink let mortgage le...,1,468
2997,thank you for shopping with us gifts for all ...,1,864
2998,the famous ebay marketing e course learn to s...,1,4129


In [26]:
spam_ham_df.groupby("label")["email_length"].describe()

# Inference - spam emails are longer than ham emails

,count,mean,std,min,25%,50%,75%,max
label,,,,,,,,
0,2500.0,1233.672000,3101.999287,5.0,360.0,719.0,1280.25,82994.0
1,499.0,2313.669339,9817.766265,1.0,525.0,956.0,1941.00,200408.0
